In [2]:
import os

In [4]:
from pyspark.sql.functions import collect_list
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import row_number, sum
from pyspark.sql.functions import col, rank

sparkSession = SparkSession.builder.enableHiveSupport().master("local[2]").getOrCreate()

data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [58]:
renamed_data = data.select(col('userId').alias('userId'),col('trackId').alias('trackId_renamed'),col('artistId').alias('artistId_renamed'),col('timestamp').alias('timestamp_renamed'))

r = data.join(renamed_data,'userId') \
.filter(col('trackId') != col('trackId_renamed')).filter((col('timestamp_renamed') - col('timestamp')) > -420).filter(((col('timestamp_renamed') - col('timestamp')) <= 420))   

r = r.groupBy('trackId','trackId_renamed').count()

w2 = Window.partitionBy('trackId').orderBy('count')

r = r.withColumn('rank',rank().over(w2))
r = r.filter(col('rank') < 40)

maxR = r.groupBy('trackId').agg(sum('count').alias('sum_count'))
r = r.join(maxR,'trackId').withColumn('normalizedWeigth',col('count') / col('sum_count'))
r = r.orderBy(col('normalizedWeigth').desc(),col('trackId'),col('trackId_renamed')).take(40)

for x in r:
    print("{} {}".format(x[0],x[1]))

798256 923706
798319 837992
798322 876562
798331 827364
798335 840741
798374 816874
798375 810685
798379 812055
798380 840113
798396 817687
798398 926302
798405 867217
798443 905923
798457 918918
798460 891840
798461 940379
798470 840814
798474 963162
798477 883244
798485 955521
798505 905671
798550 936295
798626 845438
798691 818279
798692 898823
798702 811440
798704 937570
798725 933147
798738 894170
798745 799665
798782 956938
798801 950802
798820 890393
798833 916319
798865 962662
798931 893574
798946 946408
799012 809997
799024 935246
799047 905199
